In [6]:
from jlam.py_sql import PySql

In [8]:
import jlam.conn_constants as cnst

In [9]:
sfdc=PySql(cnst.SVR_BIODS,cnst.DB_SFDC_DATA)

In [100]:
sql='''
select 
topic__c,case_type__c,subject
from dbo.[case] c
left join dbo.RecordType r on (c.recordTypeId=r.id)
where c.createddate >= '2017-01-01'
and team__c in ('Call Center 1','Call Center 2','Call Center 3','Call Center Manila','Call Center Prague','Call Center US','Call Center UK'
		,'Expense Support - Professional','Expense Solutions'
		,'Expense Support - Standard'
		,'Travel Support','Travel Solutions','Travel Support 3PTY'
		,'Invoice Support'
		,'T&E Cross Product TRIM'
		,'Expense E100 Support'
                -- R&D cases
                ,'Expense Support R&D'
                ,'Travel Support R&D'
                ,'Cross Product Support R&D')
           and topic__c in ('Travel','Expense','Invoice')            
'''

In [11]:
import pandas as pd

In [101]:
df=sfdc.fetchDF(sql)

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45160 entries, 0 to 45159
Data columns (total 3 columns):
case_type__c    45160 non-null object
subject         45160 non-null object
topic__c        45160 non-null object
dtypes: object(3)
memory usage: 1.0+ MB


In [114]:
gp=df.groupby('topic__c')

In [116]:
gp

TypeError: 'DataFrameGroupBy' object is not callable

In [106]:
f = open('subjects.txt','wb')
for idx, row in df.iterrows():
    f.write(row['subject'].encode('utf-16'))
f.close()

#### train a sentence tokenizer

In [107]:
from nltk.tokenize import PunktSentenceTokenizer

In [108]:
train_text = open('subjects.txt','rb')
custom_sent_tokenizer = PunktSentenceTokenizer(train_text)

In [51]:
from nltk.corpus import stopwords
sw = set(stopwords.words("english"))
sw.add("Profile")

In [79]:
stopwords=['profile-profile','profile-user','profile','profiles','error','user','-',"'","/","concur",'profile-user','profile-','#',"(",")"]
for x in stopwords:
    sw.add(x)

In [111]:

all_words=[]
for idx, row in df.iterrows():
    words = nltk.word_tokenize(row['subject'].lower())
    for x in words:
        if x not in sw:
            all_words.append(x)

In [81]:
len(all_words)

41911

In [83]:
tagged=nltk.pos_tag(all_words)

In [98]:
type(tagged)

list

In [95]:
nouns=[x[0] for x in tagged if x[1]=='NN' or x[1]=='NNS']

In [96]:
nouns

[u'access',
 u'assign',
 u'asst',
 u'provide',
 u'access',
 u'duplicate',
 u'login',
 u'ids',
 u'employees',
 u'self',
 u'assign',
 u'assistant',
 u'input',
 u'format',
 u'***',
 u'travel',
 u'data',
 u'case',
 u'need',
 u'login',
 u'change',
 u'loginid',
 u'ysalangre',
 u'fedmarxx.com',
 u'side',
 u'par',
 u'level',
 u'travel',
 u'book',
 u'card-we',
 u'history',
 u'boston',
 u'group',
 u'travel',
 u'vacation',
 u'reassignment',
 u'link',
 u'feed',
 u'id',
 u'link',
 u'lucas',
 u'number',
 u'settings',
 u'company',
 u'section',
 u'travel',
 u'data',
 u'integration',
 u'users',
 u'profile-working',
 u'america',
 u'password',
 u'access',
 u'travel',
 u'permission',
 u'amadeus',
 u'meal',
 u'preferences',
 u'message',
 u'ff',
 u'numbers',
 u'country',
 u'code',
 u'work',
 u'phone',
 u'numbers',
 u'james',
 u'access',
 u'profile-issues',
 u'graham',
 u'packaging',
 u'apollo',
 u'eduardo',
 u'aranda',
 u'issue',
 u'sync',
 u'details',
 u'way',
 u'notification',
 u'changes',
 u'book',
 u're

In [97]:
freq_words = nltk.FreqDist(nouns)
print freq_words.most_common(100)

[(u'travel', 836), (u'card', 675), (u'credit', 364), (u'book', 363), (u'move', 359), (u'issue', 277), (u'name', 257), (u'access', 254), (u'message', 247), (u'id', 224), (u'users', 193), (u'field', 188), (u'data', 187), (u'login', 172), (u'number', 170), (u'employee', 143), (u'trip', 137), (u'information', 136), (u'change', 134), (u'site', 134), (u'email', 132), (u'permissions', 128), (u'reservation', 127), (u'hotel', 123), (u'company', 122), (u'traveler', 121), (u'gds', 120), (u'view', 117), (u'ghost', 116), (u'permission', 113), (u'password', 111), (u'search', 104), (u'sync', 103), (u'admin', 103), (u'cards', 91), (u'case', 90), (u'flight', 90), (u'address', 85), (u'arranger', 85), (u'fop', 81), (u'support', 81), (u'account', 81), (u'star', 80), (u'calendar', 79), (u'system', 77), (u'log', 75), (u'payment', 75), (u'fields', 72), (u'please', 72), (u'air', 70), (u'passport', 69), (u'level', 69), (u'sdp', 68), (u'sr', 67), (u'option', 67), (u'profile-error', 66), (u'sabre', 66), (u'booki

In [169]:
print nltk.FreqDist(freq_words)

<FreqDist with 3828 samples and 20173 outcomes>


In [99]:
print freq_words.most_common(10)

[(u'travel', 836), (u'card', 675), (u'credit', 364), (u'book', 363), (u'move', 359), (u'issue', 277), (u'name', 257), (u'access', 254), (u'message', 247), (u'id', 224)]


In [15]:
import nltk
from nltk.tokenize import PunktSentenceTokenizer

In [16]:
tokenizer=PunktSentenceTokenizer()

In [32]:
def process_content(content):
    try:
        tokenized = tokenizer.tokenize(content)
        for i in tokenized:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            
            
            #print tagged
            #print "++++"
            
             # any adverb, any verb and required proper noun and any noun
            chunkGram = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?}"""
            
            chunkParser = nltk.RegexpParser(chunkGram)
            chunked = chunkParser.parse(tagged)
            
            print(chunked)
    except Exception as e:
        print str(e)
    

In [62]:
for idx, row in df.iterrows():
   process_content(row['subject'])

(S
  (Chunk Profile/NNP)
  -/:
  (Chunk User/NNP Profile/NNP)
  -/:
  (Chunk DOB/NNP Issue/NNP))
(S
  SR-Credit/NN
  (Chunk Cards/NNP)
  -/:
  (Chunk
    ARKADIN/NNP
    ///NNP
    NEW/NNP
    CREDIT/NNP
    CARD/NNP
    FOR/NNP
    LOW/NNP
    COST/NNP))
(S
  (Chunk SR/NNP)
  -/:
  (Chunk Profile/NNP)
  -/:
  (Chunk User/NNP administration/NN)
  -/:
  2119316//CD
  (Chunk Honda/NNP US/NNP)
  and/CC
  (Chunk CA/company/NNP)
  settings/NNS
  -/:
  mobile/NN
  not/RB
  required/VBN
  for/IN
  users/NNS)
(S
  (Chunk Profile-User/NNP Profile-profile/NNP)
  can/MD
  not/RB
  be/VB
  synchronized/VBN
  ./.)
(S (Chunk Org/NNP Unit/NNP) is/VBZ missing/VBG)
(S
  (Chunk Profile/NNP)
  -/:
  (Chunk FOP/NNP)
  -/:
  Problem/NN
  with/IN
  credit/NN
  card/NN
  inability/NN
  to/TO
  book/NN
  a/DT
  flight/NN)
(S
  (Chunk Profile/NNP -User/NNP Profile-/NNP COncur/NNP)
  add/VB
  a/DT
  user/JJ
  issue/NN)
(S
  (Chunk Profile-User/NNP)
  Profile-Users/NNS
  can/MD
  not/RB
  book/NN
  (Chunk Travel

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
case_type__c    10 non-null object
description     10 non-null object
id              10 non-null object
origin          10 non-null object
subject         10 non-null object
topic__c        10 non-null object
dtypes: object(6)
memory usage: 552.0+ bytes


In [48]:
for idx,row in df.iterrows():
    if row['topic__c'] <> None: 
        print row['id']+" - "+row['topic__c']
    else:
        print row['id']+"No Topic"
    print row['subject']
    #print process_content(row['subject'])
    print "-------------"

50032000018rD6MAAU - Expense
No response from the user.
-------------
50032000018rD6RAAU - Expense
Upload Receipt - Unknown Error Message
-------------
50032000018rd6UAAQ - Expense
More users in the expense delegates
-------------
50032000018rD6WAAU - Expense
Receipts not working in app or website
-------------
50032000018rD70AAE - Analytic Reporting
My usuario Sin Acceso a Intelligence
-------------
50032000018RD72AAG - Expense
Card charge
-------------
50032000018rd73AAA - Expense
employee upload errors
-------------
50032000018rD75AAE - Expense
System issue with imaging
-------------
50032000018rd7cAAA - Expense
Assign Credit card
-------------
50032000018rD7eAAE - Invoice
Assigning invoice results in application error
-------------
50032000018RD7hAAG - Authorization
End User: How do I remove edit locks?
-------------
50032000018rD7LAAU - Concur Request
Concur Imaging Service Not Available.
-------------
50032000017BTeMAAW - Expense
Please contact me on Report
-------------
50032000